In [50]:
# %%time
# import lxml.etree as ET
# from pathlib import Path

# data_path = Path('data')

# def comment_iter(context, *args, **kwargs):
#     c = 0
#     tags = {}
#     for event, elem in context:
#         id_ = elem.attrib['Id']
#         # post id corresponds to id of Post 
#         post_id = elem.attrib['PostId']
# #         date = elem.attrib['CreationDate'].split('T')[0]
#         score = elem.attrib['Score']
#         text = elem.attrib['Text']
# #         print(id_, post_id, score, text)
#         print(elem.attrib)
#         elem.clear()
#         for ancestor in elem.xpath('ancestor-or-self::*'):
#             while ancestor.getprevious() is not None:
#                 del ancestor.getparent()[0]
#         c += 1
#         if c == 10: break
#     del context
    

# context = ET.iterparse('data/Comments.xml', events=('end', ))
# tags = comment_iter(context)

In [38]:
!head data/Posts.xml

In [90]:
%%time
import lxml.etree as ET
from pathlib import Path
import sqlite3
import re 
import html
from datetime import datetime


name = 'StackOverflow'

connection = sqlite3.connect('{}.db'.format(name))
c = connection.cursor()


def create_table():
    c.execute("""
              CREATE TABLE IF NOT EXISTS question_answer
              (post_id TEXT PRIMARY KEY, answer_id TEXT UNIQUE, post TEXT, 
              answer TEXT, title TEXT, date DATE, score INT, tags TEXT)
              """)

sql_transactions = []
start_row = 0
cleanup = 1000000

re1 = re.compile(r'  +')

def clean_tags(x):
    x = x.replace('<', '').replace('>', ' ').strip()
    return re1.sub(' ', html.unescape(x))
    
def clean_text(x, regex=True):
    x = re.sub(r'<code>[^>]*</code>', '', x)
    x = re.sub(r'<[^>]*>', '', x)
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ').replace('\n', ' newlinechar ').replace('\r', ' newlinechar ').replace(
        '"',"'").strip()
    return re1.sub(' ', html.unescape(x))


def if_exists(elem, name, func=None):
    if name in elem.attrib:
        if func:
            return func(elem.attrib[name])
        return elem.attrib[name]
    else: return None
    
    
    
def transaction_bldr(sql, size=10):
    global sql_transactions
    sql_transactions.append(sql)
    if len(sql_transactions) > size:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transactions:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transactions = []
        
        
def test_sql(sql):
    c.execute(sql)
    connection.commit()
        

def sql_insert_has_answer(post_id, answer_id, post, answer, title, date, score, tags):
    try:
        sql = """
              INSERT INTO question_answer (post_id, answer_id, post, answer, title, date, score, tags) 
              VALUES ("{}","{}","{}","{}","{}",{},"{}");
              """.format(post_id, answer_id, post, answer, title, date, int(score), tags)
        transaction_bldr(sql)
    except Exception as e:
        print('s-ANSWER insertion',str(e))
        

def sql_insert_no_answer(post_id, answer, title, date, score, tags):
    try:
        sql = """
              INSERT INTO question_answer (post_id, answer, title, date, score, tags) 
              VALUES ("{}","{}","{}","{}",{},"{}");
              """.format(post_id, answer, title, date, int(score), tags)
#         transaction_bldr(sql)
        test_sql(sql)
    except Exception as e:
        print('s-NO-ANSWER insertion',str(e))
#         print(post_id, post, title, date, score, tags)
        

def sql_insert_replace_answer(post_id, answer_id, post, answer, title, date, score, tags):
    try:
        sql = """
              UPDATE question_answer SET post_id = ?, answer_id = ?, post = ?, answer = ?, 
              title = ?, date = ?, score = ?, tags = ? WHERE post_id = ?;
              """.format(post_id, answer_id, post, answer, title, date, score, tags, post_id)
        transaction_bldr(sql)
    except Exception as e:
        print('s-UPDATE insertion',str(e))

        
def find_post(p_id):
    try:
        sql = "SELECT answer FROM question_answer WHERE answer_id = '{}' LIMIT 1".format(p_id)
        c.execute(sql)
        output = c.fetchone()
        if output != None: return output[0]
        else: return False
    except Exception as e:
#         print(e)
        return False


def find_existing_score(p_id):
    try:
        sql = "SELECT score FROM question_answer WHERE post_id = '{}' LIMIT 1".format(p_id)
        c.execute(sql)
        output = c.fetchone()
        if output != None: return output[0]
        else: return False
    except Exception as e:
#         print(e)
        return False

def acceptable(x, cutoff=250):
    # bucket data to cutoff
    if len(x.split(' ')) > cutoff or len(x) < 1: return False
    # too long
    elif len(x) > 10000: return False
    # return everything else
    else: return True

use_best_score = False

all_tags = []
all_scores = []
def post_iter(context, *args, **kwargs):
    c = 0
#     no_tag = 0
    create_table()
    row_counter = 0
    paired_rows = 0
    for event, elem in context:
        row_counter += 1
        row_id = elem.attrib['Id']
        score = int(elem.attrib['Score'])
        date = elem.attrib['CreationDate'].split('T')[0]
        post_type_id = elem.attrib['PostTypeId']
        body = clean_text(elem.attrib['Body']) 
        
        # only on posts
        tags = if_exists(elem, 'Tags', clean_tags)
        title = if_exists(elem, 'Title')
        accepted_answer_id = if_exists(elem, 'AcceptedAnswerId')
        
        # only on answers
        parent_id = if_exists(elem, 'ParentID')
        parent_data = find_post(parent_id)
        print(parent_data)
            
#         print(parent_id, id_, parent_data, body, title, date, score, tags)
        # don't want the negative scores
        if score > 0:
#           # get rid of outlier responses 
            if acceptable(body):
#               PostTypeId is 2 for answers
                if post_type_id == '2':
#                     print('TTTTTTTTT')
                    existing_answer_score = find_existing_score(parent_id)
                    if existing_answer_score:
                        if existing_answer_score > score:
#                             print(parent_id, id_, parent_data, body, title, date, score, tags)
                            sql_insert_replace_answer(parent_id, row_id, parent_data, body, title, date, score, tags)
                else:                   
                    if parent_data:
                        paired_rows += 1
                        sql_insert_has_answer(parent_id, row_id, parent_data, body, title, date, score, tags)

                    else:
                        sql_insert_no_answer(row_id, body, title, date, score, tags)
        if row_counter % 100 == 0:
            print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

#         if row_counter > start_row:
#             if row_counter % cleanup == 0:
#                 print("Cleanin up!")
#                 sql = "DELETE FROM question_answer WHERE post IS NULL"
#                 c.execute(sql)
#                 connection.commit()
#                 c.execute("VACUUM")
#                 connection.commit()
            
        
        elem.clear()
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
        c += 1
        if c == 10: break
    del context
    

context = ET.iterparse('data/Posts.xml', events=('end', ))
tags = post_iter(context)




False
s-NO-ANSWER insertion UNIQUE constraint failed: question_answer.post_id
False
s-NO-ANSWER insertion UNIQUE constraint failed: question_answer.post_id
False
False
s-NO-ANSWER insertion UNIQUE constraint failed: question_answer.post_id
False
s-NO-ANSWER insertion UNIQUE constraint failed: question_answer.post_id
False
False
s-NO-ANSWER insertion UNIQUE constraint failed: question_answer.post_id
False
s-NO-ANSWER insertion UNIQUE constraint failed: question_answer.post_id
False
s-NO-ANSWER insertion UNIQUE constraint failed: question_answer.post_id
False
s-NO-ANSWER insertion UNIQUE constraint failed: question_answer.post_id
CPU times: user 3.11 ms, sys: 0 ns, total: 3.11 ms
Wall time: 2.35 ms


In [91]:
import pandas as pd

connection = sqlite3.connect('{}.db'.format(name))
c = connection.cursor()

df = pd.read_sql("""SELECT * FROM question_answer""", connection)
df

,post_id,answer_id,post,answer,title,date,score,tags
0,4,None,None,I want to use a track-bar to change a form's o...,"While applying opacity to a form, should we us...",2008-07-31,557,c# winforms type-conversion decimal opacity
1,6,None,None,I have an absolutely positioned containing sev...,Percentage width child element in absolutely p...,2008-07-31,249,html css css3 internet-explorer-7
2,9,None,None,"Given a representing a person's birthday, how ...",How do I calculate someone's age in C#?,2008-07-31,1745,c# .net datetime
3,11,None,None,"Given a specific value, how do I display relat...",Calculate relative time in C#,2008-07-31,1317,c# datetime time datediff relative-time-span
4,13,None,None,Is there any standard way for a Web Server to ...,Determine a User's Timezone,2008-08-01,529,javascript html browser timezone timezoneoffset
5,14,None,None,What is the difference between and in .NET? ne...,Difference between Math.Floor() and Math.Trunc...,2008-08-01,363,.net math
6,16,None,None,How do you expose a LINQ query as an ASMX web ...,Filling a DataSet or DataTable from a LINQ que...,2008-08-01,109,c# linq web-services .net-3.5
7,17,None,None,How do I store binary data in MySQL? newlinechar,Binary Data in MySQL,2008-08-01,163,mysql database binary-data data-storage


In [15]:
Id, date, score, Body 

!head data/Posts.xml

In [17]:
from collections import Counter

c = Counter(all_tags)
print(c.most_common(1000))

[('c#', 328594), ('java', 270646), ('php', 252423), ('javascript', 238425), ('jquery', 205742), ('android', 196699), ('c++', 137161), ('iphone', 135101), ('asp.net', 125212), ('python', 122467), ('.net', 122401), ('html', 112080), ('mysql', 107781), ('objective-c', 93472), ('sql', 89842), ('css', 84265), ('ios', 80490), ('ruby-on-rails', 80117), ('c', 64367), ('ruby', 50358), ('sql-server', 48640), ('wpf', 47848), ('xml', 44368), ('asp.net-mvc', 42129), ('regex', 41224), ('ajax', 40845), ('database', 37696), ('django', 35681), ('windows', 34219), ('arrays', 33721), ('xcode', 33412), ('linux', 33289), ('vb.net', 31896), ('ruby-on-rails-3', 28775), ('eclipse', 28598), ('json', 27479), ('facebook', 27357), ('winforms', 27137), ('string', 26999), ('multithreading', 26743), ('asp.net-mvc-3', 24829), ('performance', 23328), ('visual-studio-2010', 22869), ('wcf', 22573), ('image', 22243), ('linq', 21776), ('visual-studio', 21667), ('macos', 21052), ('algorithm', 20948), ('silverlight', 20728)

In [18]:
no_tag

NameError: name 'no_tag' is not defined

In [ ]:
- go through posts.xml and add all to DB
 - only save posts that have
    - good tag?
    - good text body
    - long enough
    - not too long
- then go through comments.xml and add 'best' comment that coresponds to that post id. 

In [1]:
import lxml.etree as ET
from pathlib import Path
import sqlite3
import re 
import html
from datetime import datetime

In [266]:
%%time
name = 'SO'
sql_transactions = []
start_row = 0
cleanup = 1000000

connection = sqlite3.connect('{}.db'.format(name))
c = connection.cursor()


def create_table():
    c.execute("""
              CREATE TABLE IF NOT EXISTS parent_reply
              (comment_id TEXT PRIMARY KEY, parent_id TEXT, 
              parent TEXT, comment TEXT, title TEXT, unix DATE, score INT, tags TEXT)
              """) 

    
re1 = re.compile(r'  +')

def clean_tags(x):
    x = x.replace('<', '').replace('>', ' ').replace('"',"'").strip()
    return re1.sub(' ', html.unescape(x))
    
def clean_text(x, remove_html=False):
    if remove_html:
        x = re.sub(r'<code>[^>]*</code>', '', x)
        x = re.sub(r'<[^>]*>', '', x)
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ').replace('"',"'").replace('\n', ' ').replace('\r', ' ').strip()
    return re1.sub(' ', html.unescape(x))

def if_exists(elem, name, func=None, *args):
    if name in elem.attrib:
        if func:
            return func(elem.attrib[name], *args)
        return elem.attrib[name]
    else: return None
    
    
def find_parent_title_tags(p_id):
    try:
        sql = "SELECT comment, title, tags FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(p_id)
        c.execute(sql)
        output = c.fetchone()
#         print(output[0], output[1])
        if output != None: return output[0], output[1], output[2]
        else: return (False, False, False)
    except Exception as e:
#         print(e)
        return (False, False, False)

 
def find_existing_score(p_id):
    try:
        sql = "SELECT score FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(p_id)
#         sql = "SELECT score FROM parent_reply WHERE comment_id = '{}' AND parent_id is NOT NULL LIMIT 1 ".format(p_id)
        c.execute(sql)
        output = c.fetchone()
        if output != None: return output[0]
        else: return False
    except Exception as e:
#         print(e)
        return False


def find_existing_score2(p_id, post_type_id):
    try:
        if post_type_id == '1':
            sql = "SELECT score FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(p_id)
    #         sql = "SELECT score FROM parent_reply WHERE comment_id = '{}' AND parent_id is NOT NULL LIMIT 1 ".format(p_id)
            c.execute(sql)
            output = c.fetchone()
            if output != None: return output[0]
            else: return False
        return False
    except Exception as e:
#         print(e)
        return False


# def acceptable_comment(dat, cutoff=150):
#     if 'FFFFFF' in dat: return False
# #     # bucket data to cutoff
#     elif len(dat.split()) > cutoff or len(dat) < 1: return False
# #     # too long
#     elif len(dat) > 1000: return False
# #     # return everything else
#     else: return True

    
    
def transaction_bldr(sql, size=1):
    global sql_transactions
    sql_transactions.append(sql)
    if len(sql_transactions) > size:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transactions:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transactions = []
        
        
def sql_insertion(sql):
    c.execute(sql)
    connection.commit()
    
        
def test_sql2(sql, parmas):
    c.execute(sql, parmas)
    connection.commit()
        

def sql_insert_replace_comment(commentid, parentid, parent, comment, title, time, score, tags):
    try:
        sql = """
              UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, 
              title = ?, unix = ?, score = ?, tags = ? WHERE parent_id = ?;
              """
        parmas = (parentid, commentid, parent, comment, title, time, score, tags, parentid)
        
#         print(sql)
#         transaction_bldr(sql)
        test_sql2(sql, parmas)
    except Exception as e:
        print('s-UPDATE insertion', str(e))

        
def sql_insert_has_parent(commentid, parentid, parent, comment, title, time, score, tags):
    try:
        sql = """
              INSERT INTO parent_reply (parent_id, comment_id, parent, comment, title, unix, score, tags) 
              VALUES ("{}","{}","{}","{}","{}","{}",{},"{}");
              """.format(parentid, commentid, parent, comment, title, time, int(score), tags)
#         transaction_bldr(sql)
        sql_insertion(sql)
    except Exception as e:
        print('s-PARENT insertion', str(e))

        
def sql_insert_no_parent(commentid, parentid, comment, title, time, score, tags):
    try:
        sql = """
              INSERT INTO parent_reply (parent_id, comment_id, comment, title, unix, score, tags) 
              VALUES ("{}","{}","{}","{}","{}",{},"{}");
              """.format(parentid, commentid, comment, title, time, int(score), tags)
#         transaction_bldr(sql)
        sql_insertion(sql)
    except Exception as e:
        print('s-NO-PARENT insertion', str(e) + str(parentid))

        
def post_iter(context, *args, **kwargs):
    c = 0
    row_counter = 0
    paired_rows = 0
    
    for event, elem in context:
        row_counter += 1
        comment_id = elem.attrib['Id']
        score = int(elem.attrib['Score'])
        date = elem.attrib['CreationDate'].split('T')[0]
        post_type_id = elem.attrib['PostTypeId']
#         body = clean_text(elem.attrib['Body']) 
        body = elem.attrib['Body']

        # only on posts
        tags = if_exists(elem, 'Tags', clean_tags)
        title = if_exists(elem, 'Title', clean_text)
#         accepted_answer_id = if_exists(elem, 'AcceptedAnswerId')
        # only on answers
        parent_id = if_exists(elem, 'ParentId')
        parent_data, parent_title, parent_tags = find_parent_title_tags(parent_id)

        # limit reddits based on score
        if score > 0:
#         if acceptable_comment(body):   
            body = clean_text(body) 
            existing_comment_score = find_existing_score2(parent_id, post_type_id)
            if existing_comment_score:
                if existing_comment_score > score:
#                     print('HERE')
                    sql_insert_replace_comment(comment_id, parent_id, parent_data, body, title, date, score, tags)
            else:
                if parent_data:
                    paired_rows += 1
                    sql_insert_has_parent(comment_id, parent_id, parent_data, body, parent_title, date, score, parent_tags)

                else:
                    sql_insert_no_parent(comment_id, parent_id, body, title, date, score, tags)
        if row_counter % 10000 == 0:
            print('Total Rows Read: {}, Paired Rows: {}, Time: {}'.format(row_counter, paired_rows, str(datetime.now())))

        if row_counter > start_row:
            if row_counter % cleanup == 0:
                print("Cleanin up!")
                sql = "DELETE FROM parent_reply WHERE parent IS NULL"
                c.execute(sql)
                connection.commit()
                c.execute("VACUUM")
                connection.commit()

        elem.clear()
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
        c += 1
        if c == 1000: break
        
            
    del context

if __name__ == '__main__':
    create_table()
    context = ET.iterparse('data/Posts.xml', events=('end', ))
    post_iter(context)

s-PARENT insertion near "/": syntax error
CPU times: user 770 ms, sys: 459 ms, total: 1.23 s
Wall time: 4.37 s


In [264]:
c.executescript('drop table if exists parent_reply;')

In [250]:
import pandas as pd

connection = sqlite3.connect('{}.db'.format(name))
c = connection.cursor()

df = pd.read_sql("""SELECT * FROM parent_reply""", connection)
df

,comment_id,parent_id,parent,comment,title,unix,score,tags
0,4,None,None,<p>I want to use a track-bar to change a form'...,"While applying opacity to a form, should we us...",2008-07-31,557,c# winforms type-conversion decimal opacity
1,6,None,None,<p>I have an absolutely positioned <code>div</...,Percentage width child element in absolutely p...,2008-07-31,249,html css css3 internet-explorer-7
2,7,4,<p>I want to use a track-bar to change a form'...,<p>An explicit cast to double like this isn't ...,"While applying opacity to a form, should we us...",2008-07-31,395,c# winforms type-conversion decimal opacity
3,9,None,None,<p>Given a <code>DateTime</code> representing ...,How do I calculate someone's age in C#?,2008-07-31,1745,c# .net datetime
4,11,None,None,<p>Given a specific <code>DateTime</code> valu...,Calculate relative time in C#,2008-07-31,1317,c# datetime time datediff relative-time-span
5,13,None,None,<p>Is there any standard way for a Web Server ...,Determine a User's Timezone,2008-08-01,529,javascript html browser timezone timezoneoffset
6,14,None,None,<p>What is the difference between <a href='htt...,Difference between Math.Floor() and Math.Trunc...,2008-08-01,363,.net math
7,17,None,None,<p>How do I store binary data in <a href='http...,Binary Data in MySQL,2008-08-01,163,mysql database binary-data data-storage
8,22,9,<p>Given a <code>DateTime</code> representing ...,<p>The best way that I know of because of leap...,How do I calculate someone's age in C#?,2008-08-01,26,c# .net datetime
9,24,None,None,<p>If I have a <code>trigger</code> <code>befo...,Throw an error in a MySQL trigger,2008-08-01,136,mysql database triggers


In [252]:
dat = df[(df.parent.notnull()) & (df.comment.notnull())]
print(len(dat.parent_id.unique()))
dat.head()

185


,comment_id,parent_id,parent,comment,title,unix,score,tags
2,7,4,<p>I want to use a track-bar to change a form'...,<p>An explicit cast to double like this isn't ...,"While applying opacity to a form, should we us...",2008-07-31,395,c# winforms type-conversion decimal opacity
8,22,9,<p>Given a <code>DateTime</code> representing ...,<p>The best way that I know of because of leap...,How do I calculate someone's age in C#?,2008-08-01,26,c# .net datetime
10,26,17,<p>How do I store binary data in <a href='http...,<p>The answer by phpguy is correct but I think...,Binary Data in MySQL,2008-08-01,128,mysql database binary-data data-storage
11,29,13,<p>Is there any standard way for a Web Server ...,<p>There are no HTTP headers that will report ...,Determine a User's Timezone,2008-08-01,77,javascript html browser timezone timezoneoffset
13,31,6,<p>I have an absolutely positioned <code>div</...,<p>The parent <code>div</code> needs to have a...,Percentage width child element in absolutely p...,2008-08-01,129,html css css3 internet-explorer-7


In [254]:
list(zip(dat.parent.values[120:130], dat.comment.values[120:130]))

[("<p>I wrote a windows service using VB that read some legacy data from Visual Foxpro Databases to be inserted in SQL 2005. The problem is this use to run fine in Windows server 2003 32-Bits, but the client recently moved to Windows 2003 64-Bits and now the service won't work. I'm getting a message the the VFP .NET OLEdb provider is not found. I researched and everything seems to point out that there is no solution. Any Help, please...</p>",
  "<p>Have you tried changing the target CPU to <code>x86</code> instead of <code>'Any CPU'</code> in the advanced compiler options? I know that this solves some problems with other <code>OLEDB</code> providers by forcing the use of the 32-bit version.</p>"),
 ("<p>I can get Python to work with Postgresql but I cannot get it to work with MySQL. The main problem is that on the shared hosting account I have I do not have the ability to install things such as Django or PySQL, I generally fail when installing them on my computer so maybe it's good I c

In [263]:
dat[dat.parent_id == '1873'] 

,comment_id,parent_id,parent,comment,title,unix,score,tags


In [157]:
df.comment.ix[0]

/home/matt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


"I want to use a track-bar to change a form's opacity. newlinechar newlinechar This is my code: newlinechar newlinechar newlinechar newlinechar When I build the application, it gives the following error: newlinechar newlinechar newlinechar Cannot implicitly convert type to . newlinechar newlinechar newlinechar I tried using and but then the control doesn't work. This code worked fine in a past VB.NET project. newlinechar"

In [93]:
str(None)

'None'

In [120]:
df[df.parent.notnull()]

,comment_id,parent_id,parent,comment,title,unix,score


In [199]:
z = "I've been using TortoiseSVN in a Windows environment for quite some time. It seems very feature-complete and nicely integrated into the Windows shell, and more importantly, it's fairly painless to teach to colleagues with little or no experience with source control. However, since we have moved to Windows Vista 64bit, Tortoise has been very buggy and has seemed to cause lots of explorer.exe abnormalities and crashes. This has happened both with older versions of the software and the latest version (1.5.1 build 13563). I was curious if anyone has suggestions for other Subversion clients that will run on Windows (specifically Vista 64bit). Developers here use a variety of text editors so using Visual Studio or Dreamweaver for SVN is not ideal. I have heard great things about Cornerstone, and would love something similar for Windows if it exists. I'm correlating the Vista/explorer problems with Tortoise because they normally occur when I'm using the functionality in Tortoise. Sometimes bringing up the 'merge' screen will cause the GUI to start acting very strange and eventually hang or crash. I did not see 1.5.2 -- I'm installing now, maybe that will fix some of my issues."

In [201]:
len(z)

1188

In [1]:
import pdb

import lxml.etree as ET
from pathlib import Path
import sqlite3
import re 
import html
from datetime import datetime

In [ ]:
Question-Answer ideas

- vector similarities based on average word vector
- number of matching words 
- cosine distance between TF-IDF vectors 
- Levenshtein distance

- symbolic n-grams (1 - 5)
 - SVD and take first 300 components 

# Only save each field

In [2]:
%%time
name = 'StackOverflow_newline_score'
sql_transactions = []
start_row = 0
cleanup = 1000000

connection = sqlite3.connect('{}.db'.format(name))
c = connection.cursor()

def create_table():
    c.execute("""
              CREATE TABLE IF NOT EXISTS posts
              (comment_id INT PRIMARY KEY, parent_id INT, 
              comment TEXT, title TEXT, date DATE, score INT, tags TEXT)
              """) 


re1 = re.compile(r'  +')

def clean_tags(x):
    x = x.replace('<', '').replace('>', ' ').replace('"',"'").strip()
    return re1.sub(' ', html.unescape(x))


def clean_text(x, remove_html=False):
    if remove_html:
        x = re.sub(r'<code>[^>]*</code>', '', x)
        x = re.sub(r'<[^>]*>', '', x)
#     x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
#         'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
#         '<br />', "\n").replace('\\"', '').replace('<unk>','u_n').replace(' @.@ ','.').replace(
#         ' @-@ ','-').replace('\\', ' \\ ').replace('"',"'").replace('\n', ' ').replace('\r', ' ').strip()
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ').replace('"',"'").strip()
    return re1.sub(' ', html.unescape(x))


def if_exists(elem, name, func=None, *args):
    if name in elem.attrib:
        if func:
            return func(elem.attrib[name], *args)
        return elem.attrib[name]
    else: return None
    

def acceptable_comment(dat, cutoff=150):
    if '<code>' in dat: return False
#     # bucket data to cutoff
    elif len(dat.split()) > cutoff or len(dat) < 1: return False
#     # too long
    elif len(dat) > 1000: return False
#     # return everything else
    else: return True

        
def transaction_bldr(sql, size=1000):
    global sql_transactions
    sql_transactions.append(sql)
    if len(sql_transactions) > size:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transactions:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transactions = []
        
        
def sql_insert_has_parent(commentid, parentid, comment, date, score):
    try:
        sql = """
              INSERT INTO posts (comment_id, parent_id, comment, date, score) 
              VALUES ({},{},"{}","{}",{});
              """.format(int(commentid), int(parentid), comment, date, int(score))
        transaction_bldr(sql)
    except Exception as e:
        print('s-PARENT insertion', str(e))

        
def sql_insert_no_parent(commentid, comment, title, date, score, tags):
    try:
        sql = """
              INSERT INTO posts (comment_id, comment, title, date, score, tags) 
              VALUES ({},"{}","{}","{}",{},"{}");
              """.format(int(commentid), comment, title, date, int(score), tags)
        transaction_bldr(sql)
    except Exception as e:
        print('s-NO-PARENT insertion', str(e))
        
        
def post_iter(context, *args, **kwargs):
#     c = 0
    row_counter = 0
    
    for event, elem in context:
        try:
            row_counter += 1
            comment_id = int(elem.attrib['Id'])
            score = int(elem.attrib['Score'])
            date = elem.attrib['CreationDate'].split('T')[0]
            body = elem.attrib['Body']

            # only on posts
            tags = if_exists(elem, 'Tags', clean_tags)
            title = if_exists(elem, 'Title', clean_text)
            # only on answers
            parent_id = if_exists(elem, 'ParentId')

            # limit reddits based on score
            if score > 0:
    #         if acceptable_comment(body):   
                body = clean_text(body) 
                if parent_id:
                    sql_insert_has_parent(comment_id, parent_id, body, date, score)
                else:
                    sql_insert_no_parent(comment_id, body, title, date, score, tags)
                
            if row_counter % 1000000 == 0:
                print('Total Rows Read: {}, Time: {}'.format(row_counter, str(datetime.now())))
            
        except Exception as e:
            print(str(e))
            
#         c += 1
#         if c == 10000: break

        elem.clear()
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
        
            
    del context

if __name__ == '__main__':
    create_table()
    context = ET.iterparse('data/Posts.xml', events=('end', ))
    post_iter(context)

Total Rows Read: 1000000, Time: 2018-06-23 17:57:16.062703
Total Rows Read: 2000000, Time: 2018-06-23 17:58:11.346470
Total Rows Read: 3000000, Time: 2018-06-23 17:59:04.815494
Total Rows Read: 4000000, Time: 2018-06-23 17:59:57.617083
Total Rows Read: 5000000, Time: 2018-06-23 18:00:49.738513
Total Rows Read: 6000000, Time: 2018-06-23 18:01:42.944074
Total Rows Read: 7000000, Time: 2018-06-23 18:02:38.079549
Total Rows Read: 8000000, Time: 2018-06-23 18:03:33.077172
Total Rows Read: 9000000, Time: 2018-06-23 18:04:27.374554
Total Rows Read: 10000000, Time: 2018-06-23 18:05:23.022094
Total Rows Read: 11000000, Time: 2018-06-23 18:06:19.152930
Total Rows Read: 12000000, Time: 2018-06-23 18:07:15.911589
Total Rows Read: 13000000, Time: 2018-06-23 18:08:11.348592
Total Rows Read: 14000000, Time: 2018-06-23 18:09:06.963432
Total Rows Read: 15000000, Time: 2018-06-23 18:10:01.374873
Total Rows Read: 16000000, Time: 2018-06-23 18:10:55.618402
Total Rows Read: 17000000, Time: 2018-06-23 18:11

In [3]:
import pandas as pd
# name = 'StackOverflow'

connection = sqlite3.connect('{}.db'.format(name))
c = connection.cursor()

df = pd.read_sql("""SELECT * FROM posts""", connection)
df.shape

KeyboardInterrupt: 